In [68]:
%matplotlib widget

In [69]:
import http.client
import requests
import datetime
import pandas as pd

In [70]:
import matplotlib.pyplot as plt
#plt.rcParams['figure.figsize'] = (10.0, 10.0)
#import seaborn as sns
#sns.set_context('talk')
#sns.set_style("darkgrid")


In [71]:
def ms_timestamp(dt):
    return int(dt.timestamp()*1000)

In [72]:
with open('emoncms.key') as f: key=f.read()

end = datetime.datetime.now()
start = end - datetime.timedelta(days=3)

reqmap = {
    'apikey' : key,
    'id' : 2,
    'start' : ms_timestamp(start),
    'end' : ms_timestamp(end),
    'interval' : 60
}

url = 'http://emonpi/feed/data.json'
response = requests.get(url, reqmap)

print(response.status_code)
print(response.headers)
print(response.text[:100])



200
{'Date': 'Thu, 01 Apr 2021 21:09:16 GMT', 'Server': 'Apache/2.4.38 (Raspbian)', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'Content-Length': '24231', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Content-Type': 'application/json'}
[[1617052156000,28097],[1617052216000,28108],[1617052276000,28119],[1617052336000,28131],[1617052396


In [73]:
df=pd.DataFrame([(datetime.datetime.fromtimestamp(t/1000), d) for t,d in response.json()], columns=['Time', 'Wh'])
df=df.set_index('Time')
df

,Wh
Time,
2021-03-29 22:09:16,28097
2021-03-29 22:10:16,28108
2021-03-29 22:11:16,28119
2021-03-29 22:12:16,28131
2021-03-29 22:13:16,28141
...,...
2021-04-01 22:08:54,193504
2021-04-01 22:08:59,193505
2021-04-01 22:09:04,193507


In [74]:
df.index[1] - df.index[0]

Timedelta('0 days 00:01:00')

In [75]:
ax=df.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [76]:
minute_bars = df.resample('1min').last().diff() # Wh per 1 min bar
minute_bars = minute_bars * 60 # multiply by 60 to get Watts for each bar
minute_bars = minute_bars / 1000 # divide by 1000 to convert to kWatts
minute_bars = minute_bars.rename(columns={'Wh' : 'kW'})
ax=minute_bars.plot()
ax.set_ylabel('kWatts');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [77]:
ax=minute_bars.plot.hist(bins=75)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
#Validate against meter readings
readings = [
    (pd.to_datetime("2021-03-31 10:46:00"), 122313),
    (pd.to_datetime("2021-03-31 17:39:00"), 122321)]
wh1 = df.iloc[df.index.get_loc(readings[0][0], method='nearest')].Wh
wh2 = df.iloc[df.index.get_loc(readings[1][0], method='nearest')].Wh

print(f"Emoncms thinks we used {(wh2-wh1)/1000}kWh")
print(f"Meter   thinks we used {(readings[1][1]-readings[0][1])}kWh")


Emoncms thinks we used 7.1kWh
Meter   thinks we used 8kWh
